# Project: Identify Fraud from Enron Email
### Rafael Buck


## 1. Introdução

Em 2000, Enron era uma das maiores empresas dos Estados Unidos. Já em 2002, ela colapsou e quebrou devido a uma fraude que envolveu grande parte da corporação. Resultando em uma investigação federal, muitos dados que são normalmente confidenciais, se tornaram públicos, incluindo dezenas de milhares de e-mails e detalhes financeiros para os executivos dos mais altos níveis da empresa. Neste projeto, iremos bancar detetives, e colocar nossas habilidades na construção de um modelo preditivo que visará determinar se um funcionário é ou não um funcionário de interesse (POI). Um funcionário de interesse é um funcionário que participou do escândalo da empresa Enron. Os dados financeiros e sobre e-mails dos funcionários investigados neste caso de fraude já foram previamente combinados (arquivo `"final_project_dataset.pkl"`), o que significa que eles foram indiciados, fecharam acordos com o governo, ou testemunharam em troca de imunidade no processo.

## 2. Instruções

Como está sendo utilizado o Python 3.5, foram necessárias os seguintes ajustes nos códigos de `"tester.py"`:

    print("some text") # No lugar de  print "some text"
    open(file_name, "rb") # No lugar de open(file_name, "r")
    open(file_name, "wb") # No lugar de open(file_name, "w")
    clf.fit(np.array(features_train), np.array(labels_train)) # Para a rotina funcionar com o XGBoost

Para executar o projeto, portanto:
- possuir Python 3.5 ou superior instalado
- colocar todos os arquivos em um mesmo diretório (inclusive o dataset `"final_project_dataset.pkl"`)
- executar o comando `"python poi_id.py"`

## 3. Análise

### 3.1 Objetivo do trabalho e estratégia de análise

O objetivo desse projeto é criar um modelo que, dadas certas características financeiras e não-financeiras (*features*), seja capaz de prever com precisão se um funcionário é uma *Person fo Interest* (POI) de uma fraude (*target*). Nesse projeto vamos lidar com um problema de classificação usando *Machine Learning* e aprendizagem supervisionada, pois as POIs estão sendo agrupados em duas categorias: aqueles que temos um funcionário que é um POI (valor `1.0`) e aqueles que não o são (valor `0.0`).

Como etapa de pré-processamento deste projeto, foram combinados os dados da base "Enron email and financial" em um dicionário, onde cada par chave-valor corresponde a uma pessoa. A chave do dicionário é o nome da pessoa, e o valor é outro dicionário, que contém o nome de todos os atributos e seus valores para aquela pessoa. Os atributos nos dados possuem basicamente três tipos: atributos financeiros, de email e rótulos POI (pessoa de interesse).

- **atributos financeiros**: ['salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus', 'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 'expenses', 'exercised_stock_options', 'other', 'long_term_incentive', 'restricted_stock', 'director_fees'] (todos em dólares americanos (USD))

- **atributos de email**: ['to_messages', 'email_address', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi'] (as unidades aqui são geralmente em número de emails; a exceção notável aqui é o atributo ‘email_address’, que é uma string)

- **rótulo POI**: [‘poi’] (atributo objetivo lógico (booleano), representado como um inteiro)

#### 3.1.1 Data Exploration

A primeira descoberta é que estamos trabalhando com um conjunto de dados bastante desbalanceado. No nosso dataset há 145 amostras, sendo que apenas 18 delas são classificadas como POI. Ou seja, há 12,4% de POI e 87,5% de pessoas que não são POI. Idealmente, esta proporção deve ser preservada para os novos subconjuntos de treino e teste.

Uma curiosidade foi verificar quantas amostras do dataset não possuem valores para os atributos `to_messages` ou `from_messages`. O resultado foi:
- 60 pessoas não possuem valores para os atributos `to_messages` ou `from_messages`;
- 4 delas são POI, são eles ['YEAGER F SCOTT', 'KOPPER MICHAEL J', 'HIRKO JOSEPH', 'FASTOW ANDREW S'];
- 56 delas não são POI.

Ou seja, 41,4% do dataset não possui valores para os atributos `to_messages` ou `from_messages`.

#### 3.1.2 Data Wrangling

Nessa etapa da análise, basicamente removemos os valores `NaN`, trocando-os por zeros, além de normalizar todos os zeros para o valor `0.0`. Uma coisa que chamou a atenção em 3.1.1 foi que tinha uma pessoa com o nome TOTAL na lista de pessoas que não possuem valores para os atributos `to_messages` ou `from_messages`.

Pesquisando em todos os nomes por "força bruta" mesmo, já que o dataset é pequeno, foram encontradas as seguintes "pessoas" ['TOTAL', 'THE TRAVEL AGENCY IN THE PARK']. Vamos removê-las, pois além de não serem POI, não agregarão ao nosso modelo por não representar uma pessoa real.

#### 3.1.3 Outliers

Em seguida, foram feitos alguns *scatter plots* para identificar possíveis outliers em função da feature `salary`. Em vermelho, estão as amostras que são POI e em azul as que não são POI.

<img src="salary_total_payments_outliers.png"/>
<img src="salary_bonus_outliers.png"/>
<img src="salary_total_stock_value_outliers.png"/>
<img src="salary_expenses_outliers.png"/>
<img src="salary_director_fees_outliers.png"/>

É interessante notar que as amostras com valores de salário maiores que 25M (USD) poderiam ser removidos para melhorar a análise. Isso porque o principal outlier é a amostra denominada 'TOTAL', que não se trata de um POI, tão menos de uma pessoa. Já o POI 'LAY KENNETH L' parece, em alguns momentos representar um outlier (como no gráfico `total_payments x salary`), mas em outras não. Com a seleção inteligente de características mais à frente essas dimensões em que ele aparenta ser um outlier talvez não tenha muito peso no modelo final. Por isso, vamos mantê-lo nessa análise. Abaixo os mesmos gráficos acima, contudo com a remoção dos outliers.

<img src="salary_total_payments.png"/>
<img src="salary_bonus.png"/>
<img src="salary_total_stock_value.png"/>
<img src="salary_expenses.png"/>
<img src="salary_director_fees.png"/>

Esse último gráfico é interessante, pois ele indica que se director_fee é acima de zero, pode-se induzir, pelo conjunto amostral, que ele não é um POI.

### 3.2 Features usadas para identificação das POI (Persons of Interest)

Em um primeiro momento da análise, vamos considerar todos os atributos, exceto dois deles:
- `email_address`: porque basicamente consiste de um valor único por amostra, o que é inútil para nossa análise.
- `Other`: também porque terá pouco significado em nossa análise de ML.

#### 3.2.1 Criação de novas features e Seleção inteligente de features

Aqui já é possível notar as consequências do desbalanceamento do dataset. A cada iteração do **SelectKBest** temos resultados razoavelmente diferentes. Isso porque, mesmo inserindo-se o parâmetro `'stratify=y_all'` em `'train_test_split'` durante o processo de divisão treino/teste para fazer uma amostragem estratificada, mantendo-se a proporção entre as amostras categorizadas, existe uma diferença considerável dos valores das features de algumas amostras dentro dos próprios grupos de POI e não-POI. Ou seja, se em um `'train_test_split'` acabar sendo selecionadas para treino aqueles POI com maiores salários, os resultados do SelectKBest podem ser razoavelmente diferente quando selecionamos um outro conjunto de treino em que predomina salários menores de POI. Isso, mesmo mantendo-se a proporção.

A melhor solução que se aplica aqui é executar algumas vezes o procedimento com **SelectKBest** e ver, na média, quais os melhores atributos selecionados. Outra solução, que podemos fazer em um trabalho futuro, é remover todos os maiores salários (acima de USD 1,000,000.00) e trabalhar com o Dataset resultante. Mas como a ideia da análise é identificar todos as POI, vamos mantê-los e pensar numa solução para esse desafio.

Para tentar corrigir esse desbalanceamento, vamos criar novas features que relacionam percentualmente algumas variáveis. Assim, vamos ver se conseguimos agrupar mais os grupos de POI e não POI, inclusive entre eles mesmos. Para isso, criamos as seguintes features:
- bonus_salary_ratio: bônus/salário de cada funcionário;
- total_payments_salary_ratio: total de pagamentos/salário de cada funcionário;
- total_stock_value_ratio: total de valor em ações/salário de cada funcionário.

#### Definindo o número de features

Uma pergunta que surge é: como definir o número de features a ser selecionado? Para isso, umas estratégia é utilizar o gráfico abaixo, onde selecionamos um dos algoritmos dos que vamos utilizar (Seção 3.3) e verificar o desempenho do modelo em função do número de features selecionado.

<img src="f1_vs_k.png"/>
 
Vamos selecionar umas 10 features por enquanto para comparar os diversos algoritmos de *Machine Learning*. No entanto, quando for o momento de calibrar o modelo final, vamos variar o k na região das 5-7 features para tentar obter o máximo de performance. 

Assim, executando algumas vezes o **SelectKBest** para selecionar as melhores 10 features, os resultados mais recorrentes foram:

- salary
- bonus
- loan_advances
- total_stock_value
- expenses
- exercised_stock_options
- total_payments
- from_poi_to_this_person
- total_payments_salary_ratio
- bonus_salary_ratio

#### 3.2.2 Escalonamento de features

Por fim, vamos testar o escalonamento das features selecionadas para testar os resultados finais, no momento em que validaremos o estimador e encontraremos seus melhores parâmetros. Vamos usar o **MinMaxScaler**.

### 3.3 Algoritmo selecionado

<img src="ml_map.png" width="80%"/>

Vamos testar aqui 6 algoritmos de *machine learning* para aprendizagem supervisionada considerando o fluxo do gráfico acima, o tipo de problema e a quantidade de dados do nosso Dataset. São eles: 2 bastante simples (LR e KN), 1 método que funciona muito bem com PCA (SVM), 2 métodos *ensemble*/conjunto (AB e XGB) e 1 que representa uma rede neural:
- **Logistic Regression Classifier** (LR)
    - É um dos modelos mais simples de classificação, e funciona muito bem quando os dados são **linearmente separáveis**. Pelo observado nas figuras acima, não está muito claro se os dados são linearmente separáveis, mas nesse caso não custa tentar para ver o resultado.
    <img src="logistic.png" width="50%"/>
    
- **K-Neighbors Classifier** (KN)
    - É uma classe de algoritmos especializados, denominada de *instance-based*, quando o problema exige uma objetivo bastante específico ou uma clusterização do conjunto de dados, e utiliza funções que calculam similaridade dos dados com base em uma medida de distância entre eles. Pode ser interessante pois é bastante utilizado em situações onde se busca encontrar uma anomalia, como uma fraude. Mas também, como pode ser observado na maioria das figuras, as amostras de POI acabam se misturando com as de não-POI em várias dimensões. Novamente, vale a pena analisar e ver o resultado da aplicação desse modelo.
    <img src="knn.png" width="50%"/>
    
    
- **Support Vector Machine Classifier** (SVM)
    - Esse modelo utiliza kernels, que aprendem a diferenciar duas categorias de dados com base em similaridades de exemplos passados, determinando uma borda de decisão que maximiza a distancia entre os membros mais próximos das categorias. Os exemplos de uso são parecidos com os da Regressão Logística, mas agora sem a restrição dos dados serem linearmente separáveis. Eu particularmente gosto muito desse modelo e ele sempre surpreende nos resultados, especialmente quando usado com os atributos certos e com escalonamento de features.
    <img src="svm.png" width="50%"/>
    
    
- **AdaBoost Classifier** (AB)
    - O AdaBoost é um método *ensemble*. O objetivo dos métodos *ensemble* é combinar as previsões de vários estimadores básicos, construídos com um determinado algoritmo de aprendizagem, para melhorar a generalização/robustez do modelo em um único estimador.
    - O princípio central do AdaBoost é encaixar uma sequência de estimadores fracos (ou seja, modelos que são apenas um pouco melhores do que adivinhar aleatoriamente, como pequenas árvores de decisão) em versões modificadas dos dados. As previsões de todos eles são combinadas através de uma maioria ponderada (ou soma) para produzir a previsão final. De forma iterativa, o algoritmo chega ao modelo final corrigindo os pesos entre as sequências, começando pela previsão dos dados mais fáceis e, à medida que as iterações prosseguem, focando nas amostras mais difíceis de prever.
    <img src="tree.png" width="50%"/>
    
- **XGBoost Classifier** (XGB)
    - O XGBoost é também um método *ensemble*. O XGBoost é uma implementação de árvores de decisão com *gradient boost*, projetada para velocidade e desempenho.
    
    
- **MLP (Multi-layer Perceptron) Classifier** (MLP)
    - Inspirado no funcionamento do cérebro humano, esse clasificador de muitas maneiras faz a ponte entre o aprendizado de máquina e a ciência cognitiva, sendo bastante conveniente para classificação. Seu usoé muito mais eficiente em aplicações relacionadas à *Deep Learning*, especialmente com CNN (*Convolutional Neural Network*). Sempre acho legal usar um classificador MLP para comparar com os demais, mas nesse caso será mais por diversão. Esse Classificador possui diversos parâmetros e seu uso adequado merece uma análise à parte, explorando número de camadas, estratégia na escolha de camadas, funções de ativação, número de épocas de treinamento, evitar mínimos locais.
    <img src="nn.png" width="50%"/>

#### Classificador selecionado: AdaBoost Classifier

Utilizando como critério *F1 ou F2 score* e *Recall*, o melhor algoritmo foi AdaBoost, com o seguinte resultado:

    Accuracy: 0.84507	
    Precision: 0.39384	
    Recall: 0.30050	
    F1: 0.34090	
    F2: 0.31545
    Total predictions: 15000	
    True positives:  601	False positives:  925	False negatives: 1399	True negatives: 12075
    
O segundo melhor algoritmo foi o MLP, com os seguintes resultados (mas com uma precião muito baixa, o que prejudicou o F1 e F2 score):

    Accuracy: 0.63567	
    Precision: 0.12280	
    Recall: 0.28200	
    F1: 0.17109	
    F2: 0.22393
	Total predictions: 15000	
    True positives:  564	False positives: 4029	False negatives: 1436	True negatives: 8971

Em terceiro lugar ficou o K-Neighbors, com o seguinte resultado:

    Accuracy: 0.87467	
    Precision: 0.58287	
    Recall: 0.21100	
    F1: 0.30984	
    F2: 0.24186
	Total predictions: 15000	
    True positives:  422	False positives:  302	False negatives: 1578	True negatives: 12698

A ideia aqui é selecionar um algoritmo onde otimizamos a identificação de True positives (previu que era POI e era POI) e minimizamos os False negatives (previu que não era POI, mas era POI), por isso o *Recall* e o *F1 ou F2 score* são importantes, pois queremos que o máximo de verdadeiras POIs sejam identificadas (pouco *recall*). 

Apenas como curiosidade, o XGBoost atingiu os seguintes resultados:

    Accuracy: 0.84493	
    Precision: 0.33797	
    Recall: 0.17000	
    F1: 0.22621	
    F2: 0.18876
	Total predictions: 15000	
    True positives:  340	False positives:  666	False negatives: 1660	True negatives: 12334

A regressão logística apresentou o seguinte resultado:

	Accuracy: 0.73527	
    Precision: 0.12429	
    Recall: 0.16300	
    F1: 0.14103	
    F2: 0.15344
	Total predictions: 15000	
    True positives:  326	False positives: 2297	False negatives: 1674	True negatives: 10703

O SVM não convergiu. 

#### Removendo mais outliers e analisando os resultados
E "já que estamos na chuva, vamos nos molhar". Para verificar se é bom ou não para modelagem considerar aqueles outliers que são pessoas com salários acima de USD 1,000,000.00, foi feito um teste sem eles. Então removou-se os outliers:

    my_dataset.pop('FREVERT MARK A')
    my_dataset.pop('BELDEN TIMOTHY N')
    my_dataset.pop('SKILLING JEFFREY K')
    my_dataset.pop('LAY KENNETH L')
    my_dataset.pop('LAVORATO JOHN J')

e o resultado, para o AdaBoost foi:
    
    Accuracy: 0.83336	
    Precision: 0.37103	
    Recall: 0.23950	
    F1: 0.29110	
    F2: 0.25778
	Total predictions: 14000	
    True positives:  479	False positives:  812	False negatives: 1521	True negatives: 11188
    
Para os demais, também houve queda no desempenho dos *F1 e F2 scores* e do *Recall*. Na verdade queda em todas as métricas. Ou seja, é melhor para a modelagem manter aqueles outliers que são pessoas com salários acima de USD 1,000,000.00.

#### Removendo as features criadas e analisando os resultados

Outra análise consistiu de remover as features criadas e avaliar o que acontece em nosso modelo. Assim, para o AdaBoost, ao remover as features criadas, obtivemos o seguinte resultado:

    Accuracy: 0.83973	
    Precision: 0.36763	
    Recall: 0.28050	
    F1: 0.31821	
    F2: 0.29446
	Total predictions: 15000	
    True positives:  561	False positives:  965	False negatives: 1439	True negatives: 12035
    
Se comparado ao modelo padrão do AdaBoost, há também uma perda de desempenho na generalização. Ou seja, as features criadas e selecionadas de fato agregam valor ao modelo que estamos elaborando.

### 3.4 Ajustando os parâmetros do algoritmo selecionado

#### Calibrando o Modelo (Tuning)

Como primeira etapa foi analisado qual classificador seria melhor para nosso modelo de identificação de POI. No entanto, ainda é necessário avaliar quais os melhores parâmetros para esse classificador, pois o resultado acima foi gerado para os parâmetros *default*.

Os parâmetros de um classificador podem afetar significativamente o desempenho do modelo, portanto encontrar os melhores valores para eles é importante. Por essa razão é necessário calibrar o classificador escolhido. 

#### Calibrando o AdaBoost Classifier

Assim, considerando o algoritmo selecionado, o AdaBoost, vamos ajustar os parâmetros para extrair dele o melhor estimador para solução desse problema. O AdaBoost, conforme comentado, é um método *ensemble* que usa como base uma série de árvores de decisão para formar um estimador mais robusto (uma espécie de *Random Forest*). Era de se esperar que o principal parâmetro desse método fossem árvores de decisão o que, de fato, é verdade se observar a sua documentação. 

    class sklearn.ensemble.AdaBoostClassifier(base_estimator=None, n_estimators=50, learning_rate=1.0, algorithm=’SAMME.R’, random_state=None)
    
onde 

    base_estimator : object, optional (default=DecisionTreeClassifier)
    n_estimators : integer, optional (default=50)
    learning_rate : float, optional (default=1.)
    algorithm : {‘SAMME’, ‘SAMME.R’}, optional (default=’SAMME.R’)
    random_state : int, RandomState instance or None, optional (default=None)

### 3.5 Validação

Idealmente, o modelo deve ser avaliado em amostras que não foram usadas para construir ou ajustar o modelo, de modo que forneçam uma medida imparcial de eficácia do modelo. Assim, um conjunto de amostras deve ser reservado para avaliar o modelo final. O conjunto de dados de “treinamento” é o termo geral para as amostras usadas para criar o modelo, enquanto o conjunto de dados “teste” ou “validação” é usado para qualificar o desempenho. Essa estratégia é bastante interessante e evita o *overfitting*.

#### 3.5.1 Estratégia de validação

A estratégia de validação consistirá em variar parâmtros do algoritmo AdaBoost para encontrar o melhor estimador através de *cross validation* usando *grid search*. Ela consistirá nos seguintes passos:

- Vamos apenas desconsiderar a variação do algoritmo.
- Vamos variar os parâmetros de interesse com, pelo menos, 3 valores diferentes.
- Vamos encontrar um bom estimador de Decision Tree com bom F1 score através de grid search
- Vamos encontrar um bom estimador de AdaBoost usando o estimado de Decision Tree encontrado acima junto com a variação de alguns parâmetros do método.

#### 3.5.2 Validação

O resultado da validação para a seleção da Decision Tree foi o seguinte:
    
    DecisionTreeClassifier(max_depth=10, criterion='gini')
    
Até foram testados outros parâmetros que retornaram estimadores de Decision Tree com melhores scores F1 (variando `max_depth`, `min_samples_leaf`, `class_weight`, `random_state` e `criterion`). No entanto, observou-se que ao colocar o estimador no AdaBoost, os que melhor geraram resultados aceitáveis (com *Precision* e *Recall* acima dos resultado já encontrados para o AdaBoost default). Reparou-se que Decision Trees com profundidade 10 apresentou um bom desemepnho de *Recall* e com profundidade 15 um bom desempenho com o *F1 score*

Usando então o estimador acima e variando-se `n_estimators` e `learning_rate`, encontramos o melhor estimador para o AdaBoost:

    clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10, criterion='gini'), 
                             n_estimators=75, 
                             learning_rate=.7)
                             
Os resultados atingidos por esse modelo final foram:

#### k = 10

    Accuracy: 0.81620	
    Precision: 0.33050	
    Recall: 0.36900	
    F1: 0.34869	
    F2: 0.36060
	Total predictions: 15000	
    True positives:  738	False positives: 1495	False negatives: 1262	True negatives: 11505
    
#### k = 5

	Accuracy: 0.81807	
    Precision: 0.37551	
    Recall: 0.41250	
    F1: 0.39314	
    F2: 0.40453
	Total predictions: 14000	
    True positives:  825	False positives: 1372	False negatives: 1175	True negatives: 10628

#### k = 6

    Accuracy: 0.81743	
    Precision: 0.36439	
    Recall: 0.37350	
    F1: 0.36889	
    F2: 0.37164
	Total predictions: 14000	
    True positives:  747	False positives: 1303	False negatives: 1253	True negatives: 10697

#### k = 7

    Accuracy: 0.81960	
    Precision: 0.33925	
    Recall: 0.37250	
    F1: 0.35510	
    F2: 0.36534
	Total predictions: 15000	
    True positives:  745	False positives: 1451	False negatives: 1255	True negatives: 11549
    
Nele conseguimos aumentar o *Recall* e os scores F1 e F2. 

### 3.6 Métricas utilizadas

Depois de ter construído o nosso modelo final, a questão mais importante que surge é quão bom é o modelo? Portanto, avaliar seu modelo é a tarefa mais importante no projeto de machine learning, que delineia como as nossas previsões são boas.

#### Precision (Specificity)
A precisão é a relação tp / (tp + fp) onde tp é o número de positivos verdadeiros e fp o número de falsos positivos. A precisão é intuitivamente a capacidade do classificador de não rotular como positiva uma amostra que é negativa.

#### Recall (Sensitivity)
O *recall* o é a relação tp / (tp + fn) onde tp é o número de positivos verdadeiros e fn o número de falsos negativos. O recall é intuitivamente a capacidade do classificador de encontrar todas as amostras positivas.

#### F score
A pontuação F-beta pode ser interpretada como uma média harmônica ponderada da precisão e do *recall*, em que uma pontuação F-beta atinge seu melhor valor em 1 e a pior pontuação em 0.

#### Metricas escolhidas e Resultados
Novamente, a principal métrica utilizada no Gird Search foi o *Recall* porque, conforme comentamos, queremos maximizar a identificação de True positives e minimize os False negatives, mesmo que isso custe alguns False positives. Também utilizou-se como parâmetro o *F1 e F2 score*. Para o modelo final, portanto, os resultados foram:

    Precision: 0.37551	
    Recall: 0.41250	
    F1: 0.39314	
    F2: 0.40453

As features finais escolhidas foram:

    ['salary', 'bonus', 'loan_advances', 'total_stock_value', 'expenses']

## 4. Conclusão 

Os resultados foram bons embora, no começo do projeto, achei que os scores seriam maiores. Também me supreendeu o AdaBoost e o MLP serem os melhores algoritmos. Mas no final faz bastante sentido. A forma como árvores de decisão e redes neurais definem as regiões onde estão as amostras de POI parecem se encaixar melhor que os outros algoritmos. 

Outra coisa que me surpreendeu foi que os funcionários com salários altos não são outliers. A utilização deles na modelagem foi muito positiva. O modelo final apresentou a seguinte performance:
    
    Precision: 0.37551	
    Recall: 0.41250	
    F1: 0.39314	
    F2: 0.40453

Foi uma performance boa, já que o esperado era encontrar um modelo com *Precision* e *Recall* acima de `0.3` cada.

Por fim, foi interessante como o desempenho do MLP Clasifier foi bom para resolver esse problema. Acredito que um pouco mais de análise desse método para resolver o problema de identificação de POI na Enron seria um trabalho futuro muito divertido. 

## 5. Referências

> Linear Regression Classifier: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

> K-Neighbors Classifier: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

> Support Vector Machine Classifier: http://scikit-learn.org/stable/modules/svm.html

> AdaBoost Classifier: http://scikit-learn.org/stable/modules/ensemble.html

> AdaBoost Classifier: http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html

> XGBoost Classifier: http://xgboost.readthedocs.io/en/latest/python/python_api.html

> A Gentle Introduction to XGBoost for Applied Machine Learning: https://machinelearningmastery.com/gentle-introduction-xgboost-applied-machine-learning/

> MLP (Multi Layer Perceptron) Classifier: http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

> The scoring parameter: defining model evaluation rules: http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

> What is the Difference Between Test and Validation Datasets?: https://machinelearningmastery.com/difference-test-validation-datasets/

> Precision, recall, f score and support: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html

> Pyplot tutorial: https://matplotlib.org/1.3.1/users/pyplot_tutorial.html